## Process data

In [121]:
import pandas as pd

df = pd.read_pickle("processed_df.pkl")

<IPython.core.display.Javascript object>

## Terms

In [142]:
import scattertext as st

<IPython.core.display.Javascript object>

In [143]:
import re

<IPython.core.display.Javascript object>

In [146]:
analyze_col = "requirements"

<IPython.core.display.Javascript object>

In [149]:
filtered_df = df[~df[analyze_col].isna()][["title", analyze_col, "page_host"]]

<IPython.core.display.Javascript object>

In [150]:
filtered_df["parse"] = filtered_df[analyze_col].apply(st.whitespace_nlp_with_sentences)

<IPython.core.display.Javascript object>

In [151]:
filtered_df["parse"]

0                                                      ()
1       (years, relevant, experience, preferred, healt...
2       (pursuing, phd, ms, cs, math, statistics, phys...
3                                                      ()
4       (phd, ms, computational, biology, computer, sc...
                              ...                        
1995    (monthly, team, outings, ball, games, happy, h...
1996                                                   ()
1997                                                   ()
1998    (least, years, experience, developing, busines...
1999                                                   ()
Name: parse, Length: 2000, dtype: object

<IPython.core.display.Javascript object>

In [129]:
corpus = (
    st.CorpusFromParsedDocuments(filtered_df, category_col="title", parsed_col="parse")
    .build()
    .get_unigram_corpus()
    .compact(st.AssociationCompactor(2000))
)

<IPython.core.display.Javascript object>

In [130]:
term_freq_df = corpus.get_term_freq_df()

<IPython.core.display.Javascript object>

In [131]:
import nltk

nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/khuyen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

<IPython.core.display.Javascript object>

In [132]:
term_freq_df["Data Scientist Score"] = corpus.get_scaled_f_scores("data scientist")
term_freq_df["Data Engineer Score"] = corpus.get_scaled_f_scores("data engineer")

<IPython.core.display.Javascript object>

In [133]:
def is_noun(word: str):
    pos = nltk.pos_tag([word])[0][1]
    return pos[:2] == "NN"


term_freq_df = term_freq_df.loc[map(is_noun, term_freq_df.index)]

<IPython.core.display.Javascript object>

In [134]:
term_freq_df.sort_values(by="Data Scientist Score", ascending=False).index[:30]

Index(['quantitative', 'statistics', 'r', 'machine', 'analysis', 'field',
       'science', 'ability', 'computer', 'mathematics', 'analytics', 'work',
       'techniques', 'degree', 'research', 'engineering', 'business', 'mining',
       'environment', 'time', 'knowledge', 'python', 'experience', 'problems',
       'skills', 'years', 'math', 'team', 'languages', 'bachelor'],
      dtype='object', name='term')

<IPython.core.display.Javascript object>

In [155]:
term_freq_df.sort_values(by="Data Engineer Score", ascending=False).index[:30]

Index(['apache', 'kafka', 'self', 'implement', 'spark', 'dimensional',
       'design', 'scala', 'storm', 'excellent', 'stream', 'building', 'java',
       'g', 'operating', 'management', 'flink', 'growth', 'processing',
       'hands', 'others', 'mapreduce', 'aws', 'pipelines', 'sets', 'datasets',
       'e', 'perfect', 'enemy', 'ambiguity'],
      dtype='object', name='term')


<IPython.core.display.Javascript object>

In [156]:
html = st.produce_scattertext_explorer(
    corpus,
    category="data scientist",
    category_name="Data scientist",
    not_category_name="Data Engineer",
    minimum_term_frequency=5,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    metadata=corpus.get_df()["page_host"],
    transform=st.Scalers.dense_rank,
)
open("data_science_vs_data_engineer_requirements_terms.html", "w").write(html)

1226016

<IPython.core.display.Javascript object>